# Рекомендательная система на библиотеке surprise (Домашнее задание)


In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install surprise

     |████████████████████████████████| 11.8MB 8.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670927 sha256=4ed0108506c552035e0b3288b988c8f6a2a1804811120d4b2efb0d84d8f5fc47
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [9]:
from surprise import Dataset
from surprise import Reader
from surprise import KNNBaseline
from surprise.model_selection import train_test_split

In [4]:
def Precision_at_n(df_ratings, n=3, threshold=5):
    rel_n = set(df_ratings[df_ratings.real_rating >= threshold]['itemID'])
    t = df_ratings.sort_values(by='rating', ascending=False)[0:n]
    ret_n = set(t['itemID'])
    return len(rel_n & ret_n)/n

In [5]:
def Avg_Precision_at_n(df_ratings, n=10, threshold=5):
    avg_p=0
    rel_n = set(df_ratings[df_ratings.real_rating >= threshold]['itemID'])
    
    for i in range(1,n+1):
        t=df_ratings.sort_values(by='rating', ascending=False)[0:i]
        ret_n=set(t['itemID'])
        t2=df_ratings.sort_values(by='rating', ascending=False)[i-1:i]
        ret_n2=set(t2['itemID'])
        
        if len((ret_n2 & rel_n))>0:
            avg_p=avg_p+len(rel_n & ret_n)/n
    return avg_p/n

In [6]:
def Recall_at_n(df_ratings, n=3,threshold=5):
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])
    t=df_ratings.sort_values(by='rating', ascending=False)[0:n]
    ret_n=set(t['itemID'])
    return len(rel_n & ret_n)/len(set(rel_n))

In [7]:
def MAP_at_n(df_ratings_all, n=10, threshold=5):
    users=df_ratings_all['userID'].unique()
    map_at_n=0
    for i in users:
        df_ratings=df_ratings_all[df_ratings_all.userID==i]
        map_at_n=map_at_n+Avg_Precision_at_n(df_ratings,n=n,threshold=threshold)
    return map_at_n/len(users)   

## Легкое задание
Для датасэта

```
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30)

```

Для каждого k= 10, 20 и 30 выбрать какой вариант KNN лучше item-based или user-based по метрике MAP_at_n (n=5)


In [10]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.3)

In [11]:
scores = {10: {}, 20: {}, 30: {}}

def pred_to_df(pred):
    df_ratings = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
    for i in predictions:
        df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)
    return df_ratings


for k in [10, 20, 30]:
    for based in ['user-based', 'item-based']:
        algo = KNNBaseline(k=k, sim_options={'user-based': based=='user-based'})
        predictions = algo.fit(trainset).test(testset)
        df = pred_to_df(predictions)
        scores[k][based] = MAP_at_n(df, n=5)

scores
#В обеих модификациях побеждает модель k=30
#По указанной метрике нельзя определить, что лучше: item-based или user-based

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{10: {'item-based': 0.144305408271475, 'user-based': 0.144305408271475},
 20: {'item-based': 0.15194061505832532, 'user-based': 0.15194061505832532},
 30: {'item-based': 0.15749734888653305, 'user-based': 0.15749734888653305}}

## Сложное задание

Для датасэта

```
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.30)

```

а) Для KNN(k=30) для каждого пользователя найти лучший алгоритм по Avg_Precision_at_n(7)
Cравнивая между item-based, user-based подход, а также разные меры для (косинусную, Пирсона) в KNN.
Докуметация к мерам:
https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration

б) Для каждого пользователя расчитайте рейтинги фильмов используя лучший алгоритм из пункта а)

в) Сравнить результат из б) с обычном kNN k=30 по метрике MAP (n=7).


#### а)

In [20]:
def pred_to_df(pred):
    df = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
    for i in pred:
        df = df.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)
    return df

def algo_to_df(algo):
    predictions = algo.fit(trainset).test(testset)
    df = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
    for i in predictions:
        df = df.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)
    return df

def MAP_at_n(df_ratings_all, n=10, threshold=5):
    users=df_ratings_all['userID'].unique()
    map_at_n=0
    for i in users:
        df_ratings=df_ratings_all[df_ratings_all.userID==i]
        map_at_n=map_at_n + Avg_Precision_at_n(df_ratings,n=n,threshold=threshold)
    return map_at_n/len(users)



In [21]:
algos = [
        {
            "name": "algo0",   #baseline
            "algo": KNNBaseline(k=30),
            "df": algo_to_df(KNNBaseline(k=30))
        },
        {
            "name": "algo1",
            "algo": KNNBaseline(k=30, sim_options={'user-based':True, 'name': 'cosine'}),
            "df": algo_to_df(KNNBaseline(k=30, sim_options={'user-based':True, 'name': 'cosine'}))
        },
        {
            "name": "algo2",
            "algo": KNNBaseline(k=30, sim_options={'user-based':True, 'name': 'pearson_baseline', 'shrinkage':0}),
            "df": algo_to_df(KNNBaseline(k=30, sim_options={'user-based':True, 'name': 'pearson_baseline', 'shrinkage':0}))
        },
        {
            "name": "algo3",
            "algo": KNNBaseline(k=30, sim_options={'user-based':False, 'name': 'cosine'}),
            "df": algo_to_df(KNNBaseline(k=30, sim_options={'user-based':False, 'name': 'cosine'}))
        },
        {
            "name": "algo4",
            "algo": KNNBaseline(k=30, sim_options={'user-based':False, 'name': 'pearson_baseline', 'shrinkage':0}),
            "df": algo_to_df(KNNBaseline(k=30, sim_options={'user-based':False, 'name': 'pearson_baseline', 'shrinkage':0}))           
        }
]

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [38]:
users = algos[0]['df']['userID'].unique()

def algo_user_AvgP_at_n(algo_item, userID, n=7):
    df = algo_item['df']
    df = df[df.userID == userID]
    return Avg_Precision_at_n(df, n=7)

#users df
udf = pd.DataFrame(users, columns=["userID"]).sort_values("userID")

udf["algo0_avgP7"] = np.nan
udf["algo1_avgP7"] = np.nan
udf["algo2_avgP7"] = np.nan
udf["algo3_avgP7"] = np.nan
udf["algo4_avgP7"] = np.nan

udf

,userID,algo0_avgP7,algo1_avgP7,algo2_avgP7,algo3_avgP7,algo4_avgP7
51,1,NaN,NaN,NaN,NaN,NaN
147,10,NaN,NaN,NaN,NaN,NaN
778,100,NaN,NaN,NaN,NaN,NaN
623,101,NaN,NaN,NaN,NaN,NaN
132,102,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
384,95,NaN,NaN,NaN,NaN,NaN
877,96,NaN,NaN,NaN,NaN,NaN
760,97,NaN,NaN,NaN,NaN,NaN
227,98,NaN,NaN,NaN,NaN,NaN


In [40]:
for i, row in udf.iterrows():
    userID = row.userID
    for j, algo_item in enumerate(algos):
        udf.loc[i, "algo" + str(j) + "_avgP7"] = algo_user_AvgP_at_n(algo_item, userID, n=7)

In [43]:
udf["best_algo"] = np.argmax(udf[["algo1_avgP7", "algo2_avgP7",	"algo3_avgP7", "algo4_avgP7"]].to_numpy(), axis=1) + 1

udf.set_index("userID", inplace=True)
print(len(udf))
udf.head()

943


,algo0_avgP7,algo1_avgP7,algo2_avgP7,algo3_avgP7,algo4_avgP7,best_algo
userID,,,,,,
1,0.204082,0.306122,0.204082,0.306122,0.204082,1
10,0.204082,0.122449,0.204082,0.122449,0.204082,2
100,0.000000,0.000000,0.000000,0.000000,0.000000,1
101,0.000000,0.000000,0.000000,0.000000,0.000000,1
102,0.000000,0.000000,0.000000,0.000000,0.000000,1


#### б)

In [57]:
#Теперь считаем рейтинги

df_all = pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])

for userID in users:
    best_algo = udf.loc[userID].best_algo
    df = algos[int(best_algo)]['df']
    df_all = pd.concat([df_all, df[df.userID == userID]])

#df_all.set_index('userID', inplace=True)
df_all.head()

,userID,itemID,rating,real_rating
0,262,974,2.442151,3.0
80,262,200,3.626374,3.0
329,262,195,3.689617,2.0
667,262,185,3.857548,3.0
761,262,68,3.174816,2.0


#### в)

In [55]:
### YOUR CODE HERE ###
print("MAP@7 для KNNBaseline vanilla: ", MAP_at_n(algos[0]['df'], n=7))
print("MAP@7 для составного финального датафрейма: ", MAP_at_n(df_all, n=7))

MAP@7 для KNNBaseline vanilla:  0.12541389832709296
MAP@7 для составного финального датафрейма:  0.1352825329495527
